In [9]:
import os
import cv2
import neurokit2 as nk
import numpy as np
import dlib
import matplotlib.pyplot as plt

In [10]:
root = './UBFC_DATASET/DATASET_1/'

dirs = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]

dir_name = dirs[5]
vid_folder = os.path.join(root, dir_name)

gt_filename = os.path.join(vid_folder, 'gtdump.xmp')
if os.path.isfile(gt_filename):
    gt_data = np.genfromtxt(gt_filename, delimiter=',')
    gt_time = gt_data[:, 0] / 1000
    gt_hr = gt_data[:, 1]
    gt_trace = gt_data[:, 3]
else:
    gt_filename = os.path.join(vid_folder, 'ground_truth.txt')
    if os.path.isfile(gt_filename):
        gt_data = np.loadtxt(gt_filename,)
        gt_trace = gt_data[0,:]
        gt_hr = gt_data[1,:]
        gt_time = gt_data[2,:]

gt_trace = (gt_trace - np.mean(gt_trace)) / np.std(gt_trace)

In [11]:
print("dir_name:", dir_name)
print("vid_folder:", vid_folder)
print("gt_trace shape:", gt_trace.shape)
print("gt_hr shape:", gt_hr.shape)
print("gt_time shape:", gt_time.shape)

dir_name: 7-gt
vid_folder: ./UBFC_DATASET/DATASET_1/7-gt
gt_trace shape: (5128,)
gt_hr shape: (5128,)
gt_time shape: (5128,)


In [12]:
import dlib
import cv2
vid_path = os.path.join(vid_folder, 'vid.avi')
vidObj = cv2.VideoCapture(vid_path)
face_list = []

fps = vidObj.get(cv2.CAP_PROP_FPS)
first_frame = True
tracker = dlib.correlation_tracker()

# Load the face detector and the shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_81_face_landmarks.dat")

# Function to extract facial landmarks
def extract_facial_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    landmarks = []
    for face in faces:
        shape = predictor(gray, face)
        for i in range(81):
            x = shape.part(i).x
            y = shape.part(i).y
            landmarks.append((x, y))
    return landmarks

# Def visualize points poly
def visualize_points_poly(frame, points, color, alpha=0.3):
    overlay = frame.copy()
    cv2.fillPoly(overlay, points, color)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    for i in range(0, len(points) - 1):
        cv2.line(frame, points[i], points[i+1], color, 1)

    return frame

# Open the video file
vidObj = cv2.VideoCapture(vid_path)

# Create a VideoWriter object to save the output video
output_path = os.path.join(vid_folder, 'output.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter(output_path, fourcc, fps, (int(vidObj.get(3)), int(vidObj.get(4))))

# Process each frame of the video
while True:
    ret, frame = vidObj.read()
    if not ret:
        break

    landmarks = extract_facial_landmarks(frame)
    
    # Draw landmarks on the frame
    for i, landmark in enumerate(landmarks):
        cv2.circle(frame, landmark, 1, (0, 0, 0), -1)

    frame = visualize_points_poly(frame, [np.array(landmarks)[[4, 29, 1]]], (255, 0, 0))
    frame = visualize_points_poly(frame, [np.array(landmarks)[[29, 12, 15]]], (0, 255, 0))
    frame = visualize_points_poly(frame, [np.array(landmarks)[[19, 24, 72, 69]]], (0, 0, 255))
    frame = visualize_points_poly(frame, [np.array(landmarks)[[8, 12, 57, 4]]], (255, 255, 0))

    # Write the frame to the output video
    output.write(frame)
    
    # Display the frame
    cv2.imshow('Facial Landmarks', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video objects
vidObj.release()
output.release()

# Close all windows
cv2.destroyAllWindows()